In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# Algorithme de decision tree

1. Choix de la variable racine
celle qui permet de mieux séparer les données
- teste chaque variable seule
- sélectionne celle qui donne des feuilles avec la plus faible impureté

**Feuille pure** ne contient que des 1 ou que des 0
**Feuille impure** contient un mélange de 1 et de 0

**Gini Impurity d’une feuille** = $
1 - P(\text{oui})^2 - P(\text{non})^2
$

In [3]:
df =    pd.read_csv('../data/processed/train_optimized.csv')
y = df['target']
X = df.drop(["target", "id"], axis=1)

In [4]:
X.head(1)

,keyword,text_cleaned,text_length,word_count,char_count,has_emergency_word,emergency_word_count,emergency_density,has_url,url_count,has_mention,mention_count,exclamation_count,intense_punctuation,avg_word_length,urgency_score,stopword_ratio,keyword_in_text
0,forest%20fires,a little concerned about the number of forest ...,72,14,72,False,0,0.0,False,0,False,0,0,0,4.214286,0.0,0.428571,False


In [5]:
df.head()

,id,keyword,target,text_cleaned,text_length,word_count,char_count,has_emergency_word,emergency_word_count,emergency_density,has_url,url_count,has_mention,mention_count,exclamation_count,intense_punctuation,avg_word_length,urgency_score,stopword_ratio,keyword_in_text
0,5744,forest%20fires,1,a little concerned about the number of forest ...,72,14,72,False,0,0.000000,False,0,False,0,0,0,4.214286,0.0,0.428571,False
1,4178,drown,0,when a real nigga hold you down you supposed t...,53,11,53,False,0,0.000000,False,0,False,0,0,0,3.909091,0.0,0.454545,True
2,109,accident,0,rt mention_token sleeping pills double your ri...,90,12,76,True,1,0.083333,True,1,True,1,0,0,5.416667,1.0,0.250000,True
3,5076,famine,1,new article russian food crematoria provoke ou...,123,16,107,True,1,0.062500,True,1,False,0,0,0,5.750000,0.5,0.125000,True
4,5942,hazard,0,seeing hazard without the beard like... url_token,62,7,49,False,0,0.000000,True,1,False,0,0,0,6.142857,1.0,0.142857,True


### Lister les gini impurity de chaque variable 
PAS UTILE de créer un fichier<br>
un dictionnaire en mémoire suffit


In [6]:
gini_scores = {}

def gini(group):
    n = len(group)
    if n == 0:
        return 0
    p_counts = group.value_counts(normalize=True)
    return 1 - sum(p_counts**2)

### Variables booléennes

In [7]:
def bool_lower_impurity(df, col, target='target'):

    # Sépare les True et les False et récupère les valeurs de la cible
    true_group = df[df[col]]['target']
    false_group = df[~df[col]]['target']

    total = len(df)


    true_group_gini = gini(true_group)
    false_group_gini = gini(false_group)

    # Gini impurity total pondéré (utile pour le split global si tu veux)
    weighted_gini = (
        len(true_group) / total * true_group_gini +
        len(false_group) / total * false_group_gini
    )

    return true_group_gini, false_group_gini, weighted_gini


In [8]:
for col in df.select_dtypes(include='bool'):
        gini_true, gini_false, gini_weighted = bool_lower_impurity(df, col, target='target')
        print(f"{col}\ngini(True) = {gini_true}, gini(False) = {gini_false}, gini(weighted) = {gini_weighted}\n")
        gini_scores[col] = {
        'gini_true': gini_true,
        'gini_false': gini_false,
        'gini_weighted': gini_weighted
    }

has_emergency_word
gini(True) = 0.48105339999817875, gini(False) = 0.42421029533532995, gini(weighted) = 0.4441812034084699

has_url
gini(True) = 0.49859080660310306, gini(False) = 0.41721759809750303, gini(weighted) = 0.45662143956464485

has_mention
gini(True) = 0.44089292603278607, gini(False) = 0.49245439644109057, gini(weighted) = 0.4781489019834268

keyword_in_text
gini(True) = 0.47552666614456895, gini(False) = 0.49808738115854334, gini(weighted) = 0.4807756344396553



### Variables numérique

In [9]:
def numeric_lower_impurity(df, col, target='target', n_thresholds=10):
    # Trier et choisir des seuils (ex: quantiles, ou valeurs uniques)
    unique_vals = sorted(df[col].dropna().unique())
    if len(unique_vals) <= 1:
        return None  # Rien à séparer

    thresholds = unique_vals
    if len(unique_vals) > n_thresholds:
        thresholds = np.linspace(min(unique_vals), max(unique_vals), n_thresholds + 2)[1:-1]  # n seuils équidistants

    best_gini = float('inf')
    best_threshold = None

    for thresh in thresholds:
        left = df[df[col] <= thresh][target]
        right = df[df[col] > thresh][target]
        total = len(df)

        weighted_gini = (
            len(left) / total * gini(left) +
            len(right) / total * gini(right)
        )

        if weighted_gini < best_gini:
            best_gini = weighted_gini
            best_threshold = thresh

    return best_threshold, best_gini


In [10]:
gini_scores_numeric = {}

for col in df.select_dtypes(include='number'):
    result = numeric_lower_impurity(df, col, target='target')
    if result:
        threshold, gini_value = result
        print(f"{col} : best threshold = {threshold}, gini = {gini_value}")
        gini_scores[col] = {
            'threshold': threshold,
            'gini_weighted': gini_value
        }

id : best threshold = 1978.909090909091, gini = 0.4806797910652347
target : best threshold = 0, gini = 0.0
text_length : best threshold = 75.18181818181819, gini = 0.46799164821369854
word_count : best threshold = 7.181818181818182, gini = 0.47725225987118736
char_count : best threshold = 61.0, gini = 0.473303478026487
emergency_word_count : best threshold = 0, gini = 0.4441812034084699
emergency_density : best threshold = 0.015151515151515145, gini = 0.4441812034084699
url_count : best threshold = 0, gini = 0.45662143956464485
mention_count : best threshold = 0, gini = 0.4781489019834268
exclamation_count : best threshold = 0, gini = 0.4786336466819331
intense_punctuation : best threshold = 0, gini = 0.4764827484343984
avg_word_length : best threshold = 4.8954545454545455, gini = 0.4623543816579591
urgency_score : best threshold = 1.3636363636363635, gini = 0.4800444626639192
stopword_ratio : best threshold = 0.2727272727272727, gini = 0.46849342919089737


In [ ]:
gini_scores

In [ ]:
class DecisionTreeNode:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf(self):
        return self.value is not None

In [ ]:
class DecisionTreeClassifierSimple:
            '''
            Cherche le meilleur split (booléen ou numérique) à chaque étape
            Crée récursivement les sous-arbres
            S’arrête si la profondeur max est atteinte ou si tous les labels sont identiques
            '''
            def __init__(self, max_depth=3, min_samples_split=2):
                self.max_depth = max_depth
                self.min_samples_split = min_samples_split
                self.root = None
        
            def fit(self, df, target='target', depth=0):
                y = df[target]
                if len(set(y)) == 1 or depth >= self.max_depth or len(df) < self.min_samples_split:
                    return DecisionTreeNode(value=y.mode()[0])
        
                best_gini = float('inf')
                best_col = None
                best_threshold = None
                best_sides = None
        
                for col in df.columns:
                    if col == target or col == 'id':
                        continue
                    if df[col].dtype == bool:
                        left = df[df[col] == True]
                        right = df[df[col] == False]
                        gini_left = gini(left[target])
                        gini_right = gini(right[target])
                        weighted_gini = (len(left) * gini_left + len(right) * gini_right) / len(df)
                        if weighted_gini < best_gini:
                            best_gini = weighted_gini
                            best_col = col
                            best_threshold = None
                            best_sides = (left, right)
                    elif np.issubdtype(df[col].dtype, np.number):
                        result = numeric_lower_impurity(df, col, target=target)
                        if result:
                            threshold, gini_value = result
                            left = df[df[col] <= threshold]
                            right = df[df[col] > threshold]
                            if gini_value < best_gini:
                                best_gini = gini_value
                                best_col = col
                                best_threshold = threshold
                                best_sides = (left, right)
                if best_col is None:
                    return DecisionTreeNode(value=y.mode()[0])
                left_node = self.fit(best_sides[0], target, depth+1)
                right_node = self.fit(best_sides[1], target, depth+1)
                return DecisionTreeNode(feature=best_col, threshold=best_threshold, left=left_node, right=right_node)
        
            def train(self, df, target='target'):
                self.root = self.fit(df, target)
        
            def predict_one(self, x, node=None):
                if node is None:
                    node = self.root
                if node.is_leaf():
                    return node.value
                if node.threshold is None:
                    return self.predict_one(x, node.left) if x[node.feature] else self.predict_one(x, node.right)
                else:
                    return self.predict_one(x, node.left) if x[node.feature] <= node.threshold else self.predict_one(x, node.right)
        
            def predict(self, X):
                return X.apply(lambda row: self.predict_one(row, self.root), axis=1)

In [13]:
dt = DecisionTreeClassifierSimple(max_depth=3)
dt.train(df)
preds = dt.predict(X)
print(preds.head())

0    0
1    0
2    1
3    1
4    0
dtype: int64
